## 3 Part 1: Regression Task

### 3.1 Task 1: Load and Split Dataset

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

# Load dataset
X, y = fetch_california_housing(return_X_y=True)

# Train-test split (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


### 3.2 Task 2:

### Step 1: Baseline Linear Regression (No Regularization)

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Model
lr = LinearRegression()

# Train
lr.fit(X_train, y_train)

# Predictions
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

# MSE
train_mse_lr = mean_squared_error(y_train, y_train_pred)
test_mse_lr = mean_squared_error(y_test, y_test_pred)

train_mse_lr, test_mse_lr


(0.5179331255246699, 0.5558915986952422)

### Step 2: Hyperparameter Tuning (Ridge & Lasso)

Ridge Regression (L2)

In [3]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

ridge = Ridge()

alpha_grid = {'alpha': [0.01, 0.1, 1, 10, 100]}

ridge_cv = GridSearchCV(
    ridge, alpha_grid, cv=5, scoring='neg_mean_squared_error'
)

ridge_cv.fit(X_train, y_train)

best_ridge = ridge_cv.best_estimator_

ridge_cv.best_params_


{'alpha': 10}

In [4]:
# Ridge evaluation
ridge_train_mse = mean_squared_error(y_train, best_ridge.predict(X_train))
ridge_test_mse = mean_squared_error(y_test, best_ridge.predict(X_test))

ridge_train_mse, ridge_test_mse


(0.5179417369316771, 0.5550405537342994)

Lasso Regression (L1)

In [5]:
from sklearn.linear_model import Lasso

lasso = Lasso(max_iter=10000)

lasso_cv = GridSearchCV(
    lasso, alpha_grid, cv=5, scoring='neg_mean_squared_error'
)

lasso_cv.fit(X_train, y_train)

best_lasso = lasso_cv.best_estimator_

lasso_cv.best_params_


{'alpha': 0.01}

In [6]:
# Lasso evaluation
lasso_train_mse = mean_squared_error(y_train, best_lasso.predict(X_train))
lasso_test_mse = mean_squared_error(y_test, best_lasso.predict(X_test))

lasso_train_mse, lasso_test_mse


(0.5235978023319754, 0.5444491581246521)

### Step 3: Regularization Experiments (L1 vs L2)

Coefficient Comparison

In [7]:
import numpy as np

np.sum(best_lasso.coef_ == 0), np.sum(best_ridge.coef_ == 0)


(np.int64(0), np.int64(0))

## 4 Part 2: Classification Task

### Task 1: Load and Split Dataset

In [8]:
from sklearn.datasets import load_breast_cancer

X, y = load_breast_cancer(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Step 1: Baseline Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

log_reg = LogisticRegression(max_iter=10000)

log_reg.fit(X_train, y_train)

train_acc_lr = accuracy_score(y_train, log_reg.predict(X_train))
test_acc_lr = accuracy_score(y_test, log_reg.predict(X_test))

train_acc_lr, test_acc_lr


(0.9582417582417583, 0.956140350877193)

Step 2: Hyperparameter Tuning (Logistic Regression)

In [10]:
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

log_cv = GridSearchCV(
    LogisticRegression(max_iter=10000),
    param_grid,
    cv=5,
    scoring='accuracy'
)

log_cv.fit(X_train, y_train)

log_cv.best_params_


{'C': 100, 'penalty': 'l1', 'solver': 'liblinear'}

In [11]:
best_log = log_cv.best_estimator_

train_acc_best = accuracy_score(y_train, best_log.predict(X_train))
test_acc_best = accuracy_score(y_test, best_log.predict(X_test))

train_acc_best, test_acc_best


(0.989010989010989, 0.9824561403508771)

Step 3: L1 vs L2 Logistic Regression

In [13]:
# L1
log_l1 = LogisticRegression(
    penalty='l1', C=log_cv.best_params_['C'],
    solver='liblinear', max_iter=10000
)
log_l1.fit(X_train, y_train)

# L2
log_l2 = LogisticRegression(
    penalty='l2', C=log_cv.best_params_['C'],
    solver='liblinear', max_iter=10000
)
log_l2.fit(X_train, y_train)


LogisticRegression(C=100, max_iter=10000, solver='liblinear')

In [14]:
# Accuracy comparison
accuracy_score(y_test, log_l1.predict(X_test)), accuracy_score(y_test, log_l2.predict(X_test))


(0.9824561403508771, 0.956140350877193)

In [15]:
# Sparsity check
np.sum(log_l1.coef_ == 0), np.sum(log_l2.coef_ == 0)


(np.int64(9), np.int64(0))